# Instala las librerías necesarias:

`pip install selenium beautifulsoup4`

In [9]:
!pip install selenium beautifulsoup4

Installing beautifulsoup4…
Adding beautifulsoup4 to Pipfile's [packages]…
✔ Installation Succeeded 
Pipfile.lock (ca0dcd) out of date, updating to (0b9b67)…
Locking [dev-packages] dependencies…
Locking [packages] dependencies…
✔ Success! 
Updated Pipfile.lock (ca0dcd)!
Installing dependencies from Pipfile.lock (ca0dcd)…
  🐍   ▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉▉ 76/76 — 00:00:12mmmmmmmmmmmmmmmmmmmmmmmmmmmmmmm


# Instalar Google Chrome en Linux

In [ ]:
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb

In [ ]:
!sudo dpkg -i google-chrome-stable_current_amd64.deb

# Bajar ChromeDriver

[Acá](https://chromedriver.chromium.org/downloads) puedes encontrar las diferentes versiones de ChromeDriver. Chequea tu versión de google Chrome para bajar el driver correcto

In [5]:
!google-chrome --version

Google Chrome 77.0.3865.75 


In [6]:
!ls

'1 - Introducción a Python.ipynb'   geckodriver.log   spider.py
 chromedriver			    Pipfile	      Untitled.ipynb
 chromedriver_linux64.zip	    Pipfile.lock
 domicilios			    review.json


In [11]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup

In [66]:
driver = webdriver.Chrome(executable_path="./chromedriver")
driver.get("https://domicilios.com/restaurantes/bogota/ppc-suba-menu?t=comentarios")

In [67]:
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [68]:
info = []
for review in soup.select("#reviewList > li"):
    text = review.select("[itemprop='description']")[0].get_text()
    author = review.select("div.small")[0].get_text()
    previousOrders = review.select("div.small")[1]["title"].split()[0]
    previousReviews = review.select("div.small")[2]["title"].split()[0]
    date = review.select("[itemprop='datePublished']")[0].get_text()
    worstRating = review.select("[itemprop='worstRating']")[0]["content"]
    ratingValue = review.select("[itemprop='ratingValue']")[0]["content"]
    bestRating = review.select("[itemprop='bestRating']")[0]["content"]
    infoReview = dict(text=text, author=author, previousOrders=previousOrders, previousReviews=previousReviews,
                     date=date, worstRating=worstRating, ratingValue=ratingValue, bestRating=bestRating)
    info.append(infoReview)

In [69]:
print(info)

[{'text': 'Parecía arroz del día anterior, trasnochado y de mal gusto', 'author': 'Raul fernando', 'previousOrders': '2', 'previousReviews': '0', 'date': '21 sep, 2019', 'worstRating': '1', 'ratingValue': '3.7', 'bestRating': '5'}, {'text': 'El pollo más rico que tiene colombia ❤?❤?', 'author': 'Andres  ramos', 'previousOrders': '25', 'previousReviews': '0', 'date': '25 jul, 2019', 'worstRating': '1', 'ratingValue': '5.0', 'bestRating': '5'}, {'text': 'redimi y cupón de $7.000 y me cobraron la cuenta completa sin el descuento del cupon', 'author': 'Camilo espitia', 'previousOrders': '266', 'previousReviews': '0', 'date': '20 jun, 2019', 'worstRating': '1', 'ratingValue': '4.0', 'bestRating': '5'}, {'text': 'Excelente servicio, el domiciliario es muy respetuoso y amable', 'author': 'Laura sanchez', 'previousOrders': '274', 'previousReviews': '0', 'date': '09 jun, 2019', 'worstRating': '1', 'ratingValue': '5.0', 'bestRating': '5'}, {'text': 'Gratamente Sorprendida, han Mejorado la calida

In [82]:
button = driver.find_element_by_css_selector("#reviewListContainer > footer > ul > li.arrow.next > button")
print(button.get_attribute("data-page"))
button.click()

2


In [59]:
driver.quit()

# Todo funciona perfecto! :-D

Ahora volvamos esto una función para iterar sobre todas las páginas de los comentarios de nuestro restaurante de ejemplo

In [63]:
import time

In [85]:
def download_domicilios(website):
    driver = webdriver.Chrome(executable_path="./chromedriver")
    driver.get(website)
    driver.implicitly_wait(100)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    info = []
    pages = []
    while True:
        try:
            button = driver.find_element_by_css_selector("#reviewListContainer > footer > ul > li.arrow.next > button")
            numPage = button.get_attribute("data-page")
            print('Currently at page {0}'.format(numPage))
            if numPage in pages:
                driver.quit()
                raise ValueError("numPage {0} was already visited".format(numPage))
            else:
                pages.append(numPage)
            for review in soup.select("#reviewList > li"):
                text = review.select("[itemprop='description']")[0].get_text()
                author = review.select("div.small")[0].get_text()
                previousOrders = review.select("div.small")[1]["title"].split()[0]
                previousReviews = review.select("div.small")[2]["title"].split()[0]
                date = review.select("[itemprop='datePublished']")[0].get_text()
                worstRating = review.select("[itemprop='worstRating']")[0]["content"]
                ratingValue = review.select("[itemprop='ratingValue']")[0]["content"]
                bestRating = review.select("[itemprop='bestRating']")[0]["content"]
                infoReview = dict(text=text, author=author, previousOrders=previousOrders, previousReviews=previousReviews,
                                 date=date, worstRating=worstRating, ratingValue=ratingValue, bestRating=bestRating)
                info.append(infoReview)
            time.sleep(3)
            button.click()
        except Exception as e:
            print(e)
            break
    return info

In [86]:
info = download_domicilios("https://domicilios.com/restaurantes/bogota/ppc-suba-menu?t=comentarios")

Currently at page 2
Currently at page 3
Currently at page 4
Currently at page 5
Currently at page 6
Currently at page 7
Currently at page 8
Currently at page 9
Currently at page 10
Currently at page 11
Currently at page 12
Currently at page 13
Currently at page 14
Currently at page 15
Currently at page 16
Currently at page 17
Currently at page 18
Currently at page 19
Currently at page 20
Currently at page 21
Currently at page 22
Currently at page 23
Currently at page 24
Currently at page 25
Currently at page 26
Currently at page 27
Currently at page 28
Currently at page 29
Currently at page 30
Currently at page 31
Currently at page 32
Currently at page 33
Currently at page 34
Currently at page 35
Currently at page 36
Currently at page 37
Currently at page 38
Currently at page 39
Currently at page 40
Currently at page 41
Currently at page 42
Currently at page 43
Currently at page 44
Currently at page 45
Currently at page 46
Currently at page 47
Currently at page 48
Currently at page 49


In [87]:
len(info)

1170

In [88]:
import json

with open("ppc_suba_comentarios.json", "w") as f:
    json.dump(info, f)